---
## EDA Observations Recap

From our exploratory analysis, we observed:

| # | Observation | Key Insight |
|---|-------------|-------------|
| 1 | Revenue is heavily skewed | Use growth rates, not raw values |
| 2 | Huge differences between divisions | Division is a critical feature |
| 3 | Growth trends over time | Growth rates capture meaningful patterns |
| 4 | Revenue and Expenses highly correlated | Efficiency ratio is informative |
| 5 | Financials correlate with participation | Revenue per athlete is useful |
| 6 | Many report exactly 1.0 ratio | Need data quality flag |
| 7 | Volatility varies by institution | Rolling volatility captures stability |

---
## Feature Categories

We'll organize features into logical categories:

### A. Structural Features (No Time Component)
- Division
- State

### B. Current Year Metrics
- Grand Total Revenue
- Grand Total Expenses
- Total Athletes
- Efficiency Ratio
- Revenue per Athlete

### C. Growth Features (1-Year Lookback)
- Revenue Growth 1yr
- Expense Growth 1yr

### D. Trend Features (2-Year Lookback)
- Revenue CAGR 2yr
- Expense CAGR 2yr
- Efficiency Mean 2yr

### E. Volatility Features
- Revenue Volatility 2yr
- Expense Volatility 2yr

### F. Data Quality Flags
- Reports Exactly One

---
## Feature Proposals with Justification

For each proposed feature, we document:
1. **What it is**
2. **Why we think it's useful** (linked to EDA observation)
3. **How it's calculated**
4. **Why it's safe** (no data leakage)

In [1]:
# ============================================
# FEATURE PROPOSALS
# ============================================
# We document each proposed feature with full justification

import pandas as pd

# Create a structured proposal document
feature_proposals = [
    # === STRUCTURAL FEATURES ===
    {
        'Category': 'Structural',
        'Feature': 'Division',
        'Based_On_Observation': 'Obs 2: Huge differences between D1, D2, D3',
        'Formula': 'Extract from Classification Name',
        'Why_Useful': 'D1 schools have 10x revenue of D2; fundamentally different financial profiles',
        'Why_Safe': 'Structural attribute, does not change over time, no future information',
        'Data_Needed': 'Current year classification'
    },
    {
        'Category': 'Structural',
        'Feature': 'State',
        'Based_On_Observation': 'General: Regional patterns may exist',
        'Formula': 'Use State CD directly',
        'Why_Useful': 'State funding policies and athletic culture vary by region',
        'Why_Safe': 'Structural attribute, location does not change, no future information',
        'Data_Needed': 'Current year state'
    },
    
    # === CURRENT YEAR METRICS ===
    {
        'Category': 'Current Metrics',
        'Feature': 'Grand Total Revenue',
        'Based_On_Observation': 'Obs 1: Raw values capture scale',
        'Formula': 'Use directly from data',
        'Why_Useful': 'Captures absolute size of athletic program',
        'Why_Safe': 'Current year value, available at prediction time',
        'Data_Needed': 'Year t revenue'
    },
    {
        'Category': 'Current Metrics',
        'Feature': 'Grand Total Expenses',
        'Based_On_Observation': 'Obs 1: Raw values capture scale',
        'Formula': 'Use directly from data',
        'Why_Useful': 'Captures spending level of athletic program',
        'Why_Safe': 'Current year value, available at prediction time',
        'Data_Needed': 'Year t expenses'
    },
    {
        'Category': 'Current Metrics',
        'Feature': 'Total_Athletes',
        'Based_On_Observation': 'Obs 5: Participation correlates with financials',
        'Formula': "Men's Participation + Women's Participation",
        'Why_Useful': 'Program size indicator; normalizer for per-athlete metrics',
        'Why_Safe': 'Current year value, available at prediction time',
        'Data_Needed': 'Year t participation counts'
    },
    {
        'Category': 'Current Metrics',
        'Feature': 'Efficiency_Ratio',
        'Based_On_Observation': 'Obs 4: Rev/Exp ratio captures financial health',
        'Formula': 'Revenue[t] / Expenses[t]',
        'Why_Useful': 'Values >1 = surplus, <1 = deficit; core health metric',
        'Why_Safe': 'Uses only current year data',
        'Data_Needed': 'Year t revenue and expenses'
    },
    {
        'Category': 'Current Metrics',
        'Feature': 'Revenue_Per_Athlete',
        'Based_On_Observation': 'Obs 5: Normalized metrics are useful',
        'Formula': 'Revenue[t] / Total_Athletes[t]',
        'Why_Useful': 'Normalizes revenue by program size; comparable across institutions',
        'Why_Safe': 'Uses only current year data',
        'Data_Needed': 'Year t revenue and athletes'
    },
    
    # === GROWTH FEATURES (1-YEAR) ===
    {
        'Category': 'Growth (1yr)',
        'Feature': 'Revenue_Growth_1yr',
        'Based_On_Observation': 'Obs 1 & 3: Growth rates normalize across scales',
        'Formula': '(Revenue[t] - Revenue[t-1]) / Revenue[t-1]',
        'Why_Useful': 'Captures recent momentum; positive = growing, negative = shrinking',
        'Why_Safe': 'Uses t and t-1 only (past data)',
        'Data_Needed': 'Year t and t-1 revenue'
    },
    {
        'Category': 'Growth (1yr)',
        'Feature': 'Expense_Growth_1yr',
        'Based_On_Observation': 'Obs 3: Expense growth patterns matter',
        'Formula': '(Expenses[t] - Expenses[t-1]) / Expenses[t-1]',
        'Why_Useful': 'Captures spending momentum; high expense growth could signal trouble',
        'Why_Safe': 'Uses t and t-1 only (past data)',
        'Data_Needed': 'Year t and t-1 expenses'
    },
    
    # === TREND FEATURES (2-YEAR) ===
    {
        'Category': 'Trend (2yr)',
        'Feature': 'Revenue_CAGR_2yr',
        'Based_On_Observation': 'Obs 3: Multi-year trends more stable than 1-year',
        'Formula': '(Revenue[t] / Revenue[t-2])^0.5 - 1',
        'Why_Useful': 'Compound annual growth rate; smoother than 1-year; captures trajectory',
        'Why_Safe': 'Uses t, t-1, t-2 only (past data)',
        'Data_Needed': 'Year t and t-2 revenue'
    },
    {
        'Category': 'Trend (2yr)',
        'Feature': 'Expense_CAGR_2yr',
        'Based_On_Observation': 'Obs 3: Expense trajectory matters',
        'Formula': '(Expenses[t] / Expenses[t-2])^0.5 - 1',
        'Why_Useful': 'Compound annual expense growth; signals spending trajectory',
        'Why_Safe': 'Uses t, t-1, t-2 only (past data)',
        'Data_Needed': 'Year t and t-2 expenses'
    },
    {
        'Category': 'Trend (2yr)',
        'Feature': 'Efficiency_Mean_2yr',
        'Based_On_Observation': 'Obs 4: Efficiency over time',
        'Formula': 'mean(Efficiency[t], Efficiency[t-1])',
        'Why_Useful': 'Smoothed efficiency; reduces noise from single-year anomalies',
        'Why_Safe': 'Uses t and t-1 only (past data)',
        'Data_Needed': 'Year t and t-1 efficiency ratios'
    },
    
    # === VOLATILITY FEATURES ===
    {
        'Category': 'Volatility',
        'Feature': 'Revenue_Volatility_2yr',
        'Based_On_Observation': 'Obs 7: Volatility varies by institution',
        'Formula': 'std(Revenue[t], Revenue[t-1])',
        'Why_Useful': 'High volatility = unstable finances; may predict future instability',
        'Why_Safe': 'Uses t and t-1 only (past data)',
        'Data_Needed': 'Year t and t-1 revenue'
    },
    {
        'Category': 'Volatility',
        'Feature': 'Expense_Volatility_2yr',
        'Based_On_Observation': 'Obs 7: Volatility captures stability',
        'Formula': 'std(Expenses[t], Expenses[t-1])',
        'Why_Useful': 'Expense volatility signals unpredictable spending',
        'Why_Safe': 'Uses t and t-1 only (past data)',
        'Data_Needed': 'Year t and t-1 expenses'
    },
    
    # === DATA QUALITY FLAGS ===
    {
        'Category': 'Data Quality',
        'Feature': 'Reports_Exactly_One',
        'Based_On_Observation': 'Obs 6: Many report exactly 1.0 efficiency',
        'Formula': '1 if Efficiency_Ratio == 1.0 else 0',
        'Why_Useful': 'Flags potential accounting anomaly; model can treat differently',
        'Why_Safe': 'Uses only current year data',
        'Data_Needed': 'Year t efficiency ratio'
    },
]

# Convert to DataFrame for nice display
proposals_df = pd.DataFrame(feature_proposals)
print(f"✅ Proposed {len(proposals_df)} features across {proposals_df['Category'].nunique()} categories")

✅ Proposed 15 features across 6 categories


In [2]:
# ============================================
# DISPLAY FEATURE PROPOSALS BY CATEGORY
# ============================================

for category in proposals_df['Category'].unique():
    print(f"\n{'='*80}")
    print(f"📁 CATEGORY: {category.upper()}")
    print('='*80)
    
    category_features = proposals_df[proposals_df['Category'] == category]
    
    for _, row in category_features.iterrows():
        print(f"\n🔹 {row['Feature']}")
        print(f"   Based on: {row['Based_On_Observation']}")
        print(f"   Formula:  {row['Formula']}")
        print(f"   Why useful: {row['Why_Useful']}")
        print(f"   Why safe: {row['Why_Safe']}")


📁 CATEGORY: STRUCTURAL

🔹 Division
   Based on: Obs 2: Huge differences between D1, D2, D3
   Formula:  Extract from Classification Name
   Why useful: D1 schools have 10x revenue of D2; fundamentally different financial profiles
   Why safe: Structural attribute, does not change over time, no future information

🔹 State
   Based on: General: Regional patterns may exist
   Formula:  Use State CD directly
   Why useful: State funding policies and athletic culture vary by region
   Why safe: Structural attribute, location does not change, no future information

📁 CATEGORY: CURRENT METRICS

🔹 Grand Total Revenue
   Based on: Obs 1: Raw values capture scale
   Formula:  Use directly from data
   Why useful: Captures absolute size of athletic program
   Why safe: Current year value, available at prediction time

🔹 Grand Total Expenses
   Based on: Obs 1: Raw values capture scale
   Formula:  Use directly from data
   Why useful: Captures spending level of athletic program
   Why safe: Curr

---
## ⚠️ CRITICAL: Features We Will NOT Use

It's equally important to document what we WON'T use and why.

### Forbidden Features (Cause Data Leakage)

In [3]:
# ============================================
# FORBIDDEN FEATURES
# ============================================
# These features would cause data leakage and inflate accuracy artificially

forbidden_features = [
    {
        'Feature': 'Lag1_Target_Label',
        'What_It_Is': 'Previous year\'s trajectory classification',
        'Why_Forbidden': 'Target_Label[t-1] is calculated from t-1→t transition, which overlaps with current prediction timeframe',
        'Leakage_Type': 'Target leakage'
    },
    {
        'Feature': 'Same_Trajectory_As_Lag',
        'What_It_Is': 'Whether current trajectory matches previous',
        'Why_Forbidden': 'Directly derived from Target_Label, contains target information',
        'Leakage_Type': 'Target leakage'
    },
    {
        'Feature': 'Trajectory_Changed',
        'What_It_Is': 'Whether trajectory changed from last year',
        'Why_Forbidden': 'Directly derived from Target_Label, contains target information',
        'Leakage_Type': 'Target leakage'
    },
    {
        'Feature': 'Lag1_Target_Declining/Stable/Improving',
        'What_It_Is': 'One-hot encoding of lagged target',
        'Why_Forbidden': 'Just a different representation of the leaked target',
        'Leakage_Type': 'Target leakage'
    },
    {
        'Feature': 'Future_Revenue_Growth (as feature)',
        'What_It_Is': 'Revenue growth from t to t+1',
        'Why_Forbidden': 'Uses future data (t+1) which is not available at prediction time',
        'Leakage_Type': 'Future leakage'
    },
    {
        'Feature': 'Any shift(-1) or shift(-n) feature',
        'What_It_Is': 'Any feature using future values',
        'Why_Forbidden': 'Negative shifts access future data',
        'Leakage_Type': 'Future leakage'
    },
]

print("🚫 FORBIDDEN FEATURES (WILL CAUSE DATA LEAKAGE):")
print("="*80)

for item in forbidden_features:
    print(f"\n❌ {item['Feature']}")
    print(f"   What: {item['What_It_Is']}")
    print(f"   Why forbidden: {item['Why_Forbidden']}")
    print(f"   Type: {item['Leakage_Type']}")

🚫 FORBIDDEN FEATURES (WILL CAUSE DATA LEAKAGE):

❌ Lag1_Target_Label
   What: Previous year's trajectory classification
   Why forbidden: Target_Label[t-1] is calculated from t-1→t transition, which overlaps with current prediction timeframe
   Type: Target leakage

❌ Same_Trajectory_As_Lag
   What: Whether current trajectory matches previous
   Why forbidden: Directly derived from Target_Label, contains target information
   Type: Target leakage

❌ Trajectory_Changed
   What: Whether trajectory changed from last year
   Why forbidden: Directly derived from Target_Label, contains target information
   Type: Target leakage

❌ Lag1_Target_Declining/Stable/Improving
   What: One-hot encoding of lagged target
   Why forbidden: Just a different representation of the leaked target
   Type: Target leakage

❌ Future_Revenue_Growth (as feature)
   What: Revenue growth from t to t+1
   Why forbidden: Uses future data (t+1) which is not available at prediction time
   Type: Future leakage

❌ Any 

---
## Feature Safety Verification Rules

To ensure no leakage, every feature must pass these tests:

In [4]:
# ============================================
# FEATURE SAFETY RULES
# ============================================

safety_rules = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                        FEATURE SAFETY VERIFICATION RULES                     ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  RULE 1: NO FUTURE DATA                                                      ║
║  ─────────────────────                                                       ║
║  • Features can only use data from year t or earlier (t-1, t-2, etc.)        ║
║  • Never use shift(-1), shift(-2) or any negative shift                      ║
║  • At prediction time, only past and current data is available               ║
║                                                                              ║
║  RULE 2: NO TARGET-DERIVED FEATURES                                          ║
║  ──────────────────────────────────                                          ║
║  • Never use Target_Label as input (even lagged versions)                    ║
║  • Never compute features by comparing to Target_Label                       ║
║  • The target is calculated AFTER features; features can't know the target   ║
║                                                                              ║
║  RULE 3: SHIFT DIRECTION CHECK                                               ║
║  ────────────────────────────────                                            ║
║  • shift(1) = look at PREVIOUS year → ✅ SAFE                                ║
║  • shift(2) = look at 2 years ago → ✅ SAFE                                  ║
║  • shift(-1) = look at NEXT year → ❌ FORBIDDEN                              ║
║  • shift(-2) = look at 2 years ahead → ❌ FORBIDDEN                          ║
║                                                                              ║
║  RULE 4: NAME CHECK                                                          ║
║  ────────────────────                                                        ║
║  • If a feature name contains "Target" → ❌ FORBIDDEN                        ║
║  • If a feature name contains "Future" → ❌ INVESTIGATE                      ║
║  • Only the actual target column should reference "Target"                   ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(safety_rules)


╔══════════════════════════════════════════════════════════════════════════════╗
║                        FEATURE SAFETY VERIFICATION RULES                     ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  RULE 1: NO FUTURE DATA                                                      ║
║  ─────────────────────                                                       ║
║  • Features can only use data from year t or earlier (t-1, t-2, etc.)        ║
║  • Never use shift(-1), shift(-2) or any negative shift                      ║
║  • At prediction time, only past and current data is available               ║
║                                                                              ║
║  RULE 2: NO TARGET-DERIVED FEATURES                                          ║
║  ──────────────────────────────────                                          ║
║  • Never use Target_Label

---
## Final Feature List Summary

In [5]:
# ============================================
# FINAL FEATURE LIST
# ============================================

final_features = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                        FINAL FEATURE LIST (15 Features)                      ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  STRUCTURAL FEATURES (2):                                                    ║
║  ├─ Division (D1/D2/D3)                                                      ║
║  └─ State                                                                    ║
║                                                                              ║
║  CURRENT METRICS (5):                                                        ║
║  ├─ Grand Total Revenue                                                      ║
║  ├─ Grand Total Expenses                                                     ║
║  ├─ Total_Athletes                                                           ║
║  ├─ Efficiency_Ratio                                                         ║
║  └─ Revenue_Per_Athlete                                                      ║
║                                                                              ║
║  GROWTH FEATURES - 1 YEAR (2):                                               ║
║  ├─ Revenue_Growth_1yr                                                       ║
║  └─ Expense_Growth_1yr                                                       ║
║                                                                              ║
║  TREND FEATURES - 2 YEAR (3):                                                ║
║  ├─ Revenue_CAGR_2yr                                                         ║
║  ├─ Expense_CAGR_2yr                                                         ║
║  └─ Efficiency_Mean_2yr                                                      ║
║                                                                              ║
║  VOLATILITY FEATURES (2):                                                    ║
║  ├─ Revenue_Volatility_2yr                                                   ║
║  └─ Expense_Volatility_2yr                                                   ║
║                                                                              ║
║  DATA QUALITY FLAGS (1):                                                     ║
║  └─ Reports_Exactly_One                                                      ║
║                                                                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  IDENTIFIER COLUMNS (kept for reference, not used in model):                 ║
║  ├─ UNITID                                                                   ║
║  ├─ Institution_Name                                                         ║
║  └─ Year                                                                     ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(final_features)


╔══════════════════════════════════════════════════════════════════════════════╗
║                        FINAL FEATURE LIST (15 Features)                      ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  STRUCTURAL FEATURES (2):                                                    ║
║  ├─ Division (D1/D2/D3)                                                      ║
║  └─ State                                                                    ║
║                                                                              ║
║  CURRENT METRICS (5):                                                        ║
║  ├─ Grand Total Revenue                                                      ║
║  ├─ Grand Total Expenses                                                     ║
║  ├─ Total_Athletes                                                           ║
║  ├─ Efficiency_Ratio     

---
## Year Allocation Reminder

When we build these features in Notebook 04, we'll lose some years to lag calculations:

In [6]:
# ============================================
# YEAR ALLOCATION
# ============================================

year_allocation = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                             YEAR ALLOCATION                                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  Raw Data: 2014-2023 (10 years)                                              ║
║                                                                              ║
║  2014    2015    2016    2017    2018    2019    2020    2021    2022    2023║
║  |_______|_______|       |_______|_______|_______|_______|_______|_______|   ║
║                                                                              ║
║  LOOKBACK (consumed)     TRAIN           VALIDATION      HOLDOUT   FUTURE   ║
║  for 2-year features     (learn)         (tune)          (test)    (predict)║
║                                                                              ║
║  Why 2014-2016 are consumed:                                                 ║
║  • To calculate Revenue_CAGR_2yr for 2017, we need 2015 and 2017 data        ║
║  • To calculate Revenue_Growth_1yr for 2017, we need 2016 data               ║
║  • First usable row is 2017 (has 2-year history available)                   ║
║                                                                              ║
║  Why 2023 has no target:                                                     ║
║  • Target is calculated from t→t+1 transition                                ║
║  • For 2023, we would need 2024 data (doesn't exist)                         ║
║  • 2023 becomes our TRUE FUTURE PREDICTION set                               ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(year_allocation)


╔══════════════════════════════════════════════════════════════════════════════╗
║                             YEAR ALLOCATION                                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  Raw Data: 2014-2023 (10 years)                                              ║
║                                                                              ║
║  2014    2015    2016    2017    2018    2019    2020    2021    2022    2023║
║  |_______|_______|       |_______|_______|_______|_______|_______|_______|   ║
║                                                                              ║
║  LOOKBACK (consumed)     TRAIN           VALIDATION      HOLDOUT   FUTURE   ║
║  for 2-year features     (learn)         (tune)          (test)    (predict)║
║                                                                              ║
║  Why 2014-2016 are consumed

---
## ✅ Checkpoint Complete

We've documented:
1. **15 proposed features** across 6 categories
2. **Justification** for each feature linked to EDA observations
3. **Safety verification** that each feature uses only past data
4. **Forbidden features** that would cause leakage
5. **Year allocation** showing which years are used for what

**Next Notebook:** `03_Target_Definition.ipynb`  
Before building features, we need to clearly define what we're predicting (the target variable).